In [1]:
!pip install -q langchain-community langchain-huggingface \ faiss-cpu python-dotenv tiktoken sentence-transformers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

In [ ]:
HUGGINGFACEHUB_API_TOKEN = "hf____"

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
import json

In [4]:
from huggingface_hub import login
login() # You will be prompted for your HF key, which will then be saved locally

## Step 1a- Indexing (Document Ingestion)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Path to the file in your Drive
file_path = "/content/drive/MyDrive/Nugget/combined_restaurants.txt"

with open(file_path, 'r') as file:
    text_data = file.read()

print(text_data[:500])

# aryabhavan.txt
Aryabhavan Restaurant has description -- Arya Bhavan offers a wide selection of South India Dishes for our esteemed patrons. All the South Indian delicacies are prepared by our expert chefs who use South Indian ingredients. Step into a tasteful journey where our chefs bring the flavors of the south onto your tables at Arya Bhavan.Indulge in South Indian delicacies like the variety of Dosa’s, Malabar paratha, Idli, Vada at Arya Bhavan – one of the top-rated vegetarian restaurants


## Step 1b - Indexing (Text Splitting)

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [14]:
chunks = text_splitter.create_documents([text_data])

## Step 1c & 1d - Indexing (Embedding Generation and Storing Vector Store)

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS

# Use a local embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create a FAISS vector store using the same chunks
vector_store = FAISS.from_documents(chunks, embeddings)

In [16]:
vector_store.index_to_docstore_id

{0: '32dc531c-de76-4570-8b51-d15700d9a776',
 1: '78cf2ad1-8a4a-433e-b992-13b829ce0cd2',
 2: 'f20f36a6-5aad-44c5-851f-19b8058545fc',
 3: '3aff59f4-bc35-40c2-8837-ed9c3cc3e194',
 4: '4a500ea5-75e2-47ce-bec1-947f33e87d4d',
 5: '02e586ce-a238-4cef-9537-43910d1da45c',
 6: '6a1dbdd0-2bd0-4888-a537-cd69e48e8563',
 7: 'e2597f6b-57f6-4272-bdf2-f44fc6608e4a',
 8: 'e7282c96-1dd3-4833-88a7-40fa414d7e96',
 9: '939d31ab-31f4-4e4a-b23e-207894ff4fb0',
 10: '01e6d820-0a28-4579-8db4-cf383dc5c766',
 11: '6298e7ef-349e-4d56-88ee-e3805eec1136',
 12: '4273d656-f2fc-4078-835b-2df349877323',
 13: 'bc9253df-6fee-408f-92a8-03c30837e7cb',
 14: 'e1a371ee-3b8e-46f3-bf3a-c3e434f072f3',
 15: '569937f0-c0d6-412e-8540-2a4c26747bb9',
 16: '3564b37d-58ca-4b1e-bd1d-943262299645',
 17: 'e8a00218-b9f2-4b3c-8cae-deb39d915a78',
 18: 'a3440177-c27f-48f8-afd8-a8e6e73b4291',
 19: 'f565ad32-b86f-4cb5-8575-82281ed11150',
 20: 'ed9d3ba7-ae79-4448-a830-93584a3a3876',
 21: 'db8b4ee4-c791-43cb-a84e-bb49ecb536f3',
 22: 'd4fd190c-cb2b-

In [17]:
vector_store.get_by_ids(['5858d8a2-f00c-4f03-a4f7-a8f2747f1c37'])

[Document(id='5858d8a2-f00c-4f03-a4f7-a8f2747f1c37', metadata={}, page_content='Description: little gem lettuce, pepperoni, chickpeas, green olives, pistachios, oregano, red wine vinaigrette, pecorino\nPastaria St. Louis Restaurant has menu item, Shaved Kale in Soups and Salads priced at 11.95 dollars.\nDescription: creamy anchovy dressing, pecorino, bread crumbs\nPastaria St. Louis Restaurant has menu item, Arugula & Endive in Soups and Salads priced at 11.95 dollars.\nDescription: white wine vinaigrette, dried cherry, pistachio, goat cheese, breadcrumbs\nPastaria St. Louis Restaurant has menu item, Brussels Sprouts in Vegetables priced at 10.95 dollars.\nDescription: lemon, calabrian chili flake, garlic\nPastaria St. Louis Restaurant has menu item, Roasted Heirloom Carrots in Vegetables priced at 9.95 dollars.\nDescription: confit leeks, herbs, toasted breadcrumb\nPastaria St. Louis Restaurant has menu item, Crispy Risotto Balls in Appetizers priced at 12.95 dollars.\nDescription: mo

## Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [19]:
retriever.invoke('What add-on options are available for BURRATA at Smith & Wollensky and what are their prices?')

[Document(id='f45cc2f6-1aa7-4152-90a5-c8e980aa20f2', metadata={}, page_content="Smith & Wollensky Restaurant has menu item HAMACHI CRUDO in category STARTERS priced at $26.00.\nHAMACHI CRUDO description: citrus, green apple, cucumber, tomato, passion fruit sauce\nSmith & Wollensky Restaurant has menu item AMERICAN WAGYU CARPACCIO* in category STARTERS priced at $27.00.\nAMERICAN WAGYU CARPACCIO* description: SRF Wagyu tenderloin, pickled mushrooms, black truffle aioli, shaved Parmesan\nSmith & Wollensky Restaurant has menu item BURRATA in category STARTERS priced at $19.00.\nBURRATA description: herb pesto, heirloom tomato, Marcona almond\nSmith & Wollensky Restaurant's BURRATA add-on add prosciutto at $6.00\nSmith & Wollensky Restaurant has menu item JUMBO SHRIMP COCKTAIL in category SHELLFISH priced at .\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 3 pieces at $19.00\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 5 pieces at $30.00"),
 Document(id='95917

## Step 3 - Augmentation

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# hf_model = pipeline(
#     model="google/flan-t5-small",
#     task="text2text-generation",
#     device=0,
#     max_new_tokens=64,
#     temperature=0.7
# )

hf_model = pipeline(
    "text-generation",
    model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",  # for example
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=True,
    max_new_tokens=128,
    temperature=0.2
)

llm = HuggingFacePipeline(pipeline=hf_model)

## Works but less tokens

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [26]:
from langchain_huggingface import HuggingFaceEndpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={"max_length": 128},
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

## Does not work

In [33]:
from langchain_huggingface import HuggingFaceEndpoint
repo_id = "distilbert/distilgpt2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={"max_length": 128},
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

# Input validation error: `inputs` tokens + `max_new_tokens` must be <= 1024. Given: 1248 `inputs` tokens and 512 `max_new_tokens`
# {"error":"Input validation error: `inputs` tokens + `max_new_tokens` must be <= 1024. Given: 1248 `inputs` tokens and 512 `max_new_tokens`","error_type":"validation"}

In [39]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-small",
    task="text-generation",               # generic text‐generation pipeline
    model_kwargs={"max_new_tokens":64,    # ≤ model’s max (e.g. 250)
                  "temperature":0.7},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

# Works

<ipython-input-39-31936d142683>:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [40]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided restaurant context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [41]:
question          = "What add-on options are available for BURRATA at Smith & Wollensky and what are their prices?"
retrieved_docs    = retriever.invoke(question)

In [42]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"Smith & Wollensky Restaurant has menu item HAMACHI CRUDO in category STARTERS priced at $26.00.\nHAMACHI CRUDO description: citrus, green apple, cucumber, tomato, passion fruit sauce\nSmith & Wollensky Restaurant has menu item AMERICAN WAGYU CARPACCIO* in category STARTERS priced at $27.00.\nAMERICAN WAGYU CARPACCIO* description: SRF Wagyu tenderloin, pickled mushrooms, black truffle aioli, shaved Parmesan\nSmith & Wollensky Restaurant has menu item BURRATA in category STARTERS priced at $19.00.\nBURRATA description: herb pesto, heirloom tomato, Marcona almond\nSmith & Wollensky Restaurant's BURRATA add-on add prosciutto at $6.00\nSmith & Wollensky Restaurant has menu item JUMBO SHRIMP COCKTAIL in category SHELLFISH priced at .\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 3 pieces at $19.00\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 5 pieces at $30.00\n\nSmith & Wollensky Restaurant has menu item USDA PRIME DRY-AGED PORTERHOUSE FOR TWO - 42 OZ in cate

In [43]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

## Step 4 - Generation

In [44]:
answer = llm.invoke(final_prompt)
print("answer is:", answer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


answer is: add prosciutto at $6.00


## Building a Chain

In [45]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [46]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [47]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [48]:
parallel_chain.invoke('What add-on options are available for BURRATA at Smith & Wollensky and what are their prices?')

{'context': "Smith & Wollensky Restaurant has menu item HAMACHI CRUDO in category STARTERS priced at $26.00.\nHAMACHI CRUDO description: citrus, green apple, cucumber, tomato, passion fruit sauce\nSmith & Wollensky Restaurant has menu item AMERICAN WAGYU CARPACCIO* in category STARTERS priced at $27.00.\nAMERICAN WAGYU CARPACCIO* description: SRF Wagyu tenderloin, pickled mushrooms, black truffle aioli, shaved Parmesan\nSmith & Wollensky Restaurant has menu item BURRATA in category STARTERS priced at $19.00.\nBURRATA description: herb pesto, heirloom tomato, Marcona almond\nSmith & Wollensky Restaurant's BURRATA add-on add prosciutto at $6.00\nSmith & Wollensky Restaurant has menu item JUMBO SHRIMP COCKTAIL in category SHELLFISH priced at .\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 3 pieces at $19.00\nSmith & Wollensky Restaurant's JUMBO SHRIMP COCKTAIL add-on 5 pieces at $30.00\n\nSmith & Wollensky Restaurant has menu item USDA PRIME DRY-AGED PORTERHOUSE FOR TWO - 4

In [49]:
parser = StrOutputParser()

In [50]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('What add-on options are available for BURRATA at Smith & Wollensky and what are their prices?')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'add prosciutto at $6.00'